In [9]:
def t_cost(s,t): return ((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5)
def d_cost(s,t): return alpha*(((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5))
alpha =0.5

In [10]:
import itertools
import matplotlib.pyplot as plt
def two_opt(ps):
    n = len(ps)
    res = list(range(n))
    update = True
    while update:
        update = False
        for (i,j) in itertools.combinations(range(n),2):
            if ((t_cost(ps[res[i]],ps[res[i+1]])+t_cost(ps[res[j]],ps[res[(j+1)%n]]))>
                (t_cost(ps[res[i]],ps[res[j]])+t_cost(ps[res[i+1]],ps[res[(j+1)%n]]))):
                res[i+1:j+1] = res[j:i:-1]
                update = True
                
#     for i in range(n):
#         plt.scatter(ps[i][0],ps[i][1],c='k')
#         connect_x=[ps[res[i]][0],ps[res[(i+1)%len(ps)]][0]]
#         connect_y=[ps[res[i]][1],ps[res[(i+1)%len(ps)]][1]]
#         plt.plot(connect_x,connect_y, c='red')
    return res

In [11]:
def t_pathcost(a,b,tsp):
    pathcost=0
    if a<b:
        for i in range(a,b):
            pathcost=pathcost+t_cost(V[tsp[i]],V[tsp[(i+1)%n]])
    if a>b:
        for i in range(a,len(tsp)):
            pathcost=pathcost+t_cost(V[tsp[i]],V[tsp[(i+1)%len(tsp)]])
        for i in range(b):
            pathcost=pathcost+t_cost(V[tsp[i]],V[tsp[(i+1)%len(tsp)]])
    if a==b:
        pathcost=0
            
    return pathcost

In [23]:
import itertools
def heuristic2(tsp):
    T={}
    M={}
    D={}
    Mdrone={}
    route={}
    n=len(tsp)
    for (start,drone,end) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[drone+1]])+t_pathcost(drone+1,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
    for (end,start,drone) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[(drone+1)%n]])+t_pathcost((drone+1)%n,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
    for (drone,end,start) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[(drone+1)%n]])+t_pathcost((drone+1)%n,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
        
    for (i,j) in itertools.permutations(range(n),2):
        M[i,j]=float('inf')
        if i<j:
            if i+1==j:
                M[i,j]=t_cost(V[tsp[i]],V[tsp[j]])
            for k in range(i+1,j):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[i,j]=[k]
        else:
            for k in range(i+1,n):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[i,j]=[k]
            for k in range(0,j):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[i,j]=[k]
            M[n-1,0]=t_cost(V[tsp[n-1]],V[tsp[0]])

    for depot in range(n):
        for i in range(depot,depot+n+1):
            D[depot,i%n]=float('inf')
            if i==depot:
                D[depot,i]=0
                route[depot,i%n]=[[depot]]
            else:
                for j in range(depot,i):
                    if j!=i%n:
                        d=M[j%n,i%n]+D[depot,j%n]
                        r=route[depot,j%n]+[[j%n,i%n]]
                        if D[depot,i%n]>d:
                                D[depot,i%n]=d
                                route[depot,i%n]=r
        
    res=[D[i,i] for i in range(n)]
    dep=res.index(min(res))
    pathlen=len(route[dep,dep])
    path=[]#通る順番（点の座標ではなくインデックスを表している点に注意）
    for i in range(pathlen-1):
        path=path+route[dep,dep][i+1]
        if (route[dep,dep][i+1][0],route[dep,dep][i+1][1]) in Mdrone:
            path=path+[Mdrone[route[dep,dep][i+1][0],route[dep,dep][i+1][1]]]
    
        
    
    return min(res)

In [26]:
import itertools
def heuristic2original(tsp):
    T={}
    M={}
    D={}
    Mdrone={}
    route={}
    n=len(tsp)
    for (start,drone,end) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[drone+1]])+t_pathcost(drone+1,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
#     for (end,start,drone) in itertools.combinations(range(n),3):
#         drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
#         truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[(drone+1)%n]])+t_pathcost((drone+1)%n,end,tsp)
#         T[start,drone,end]=max(drone_cost,truck_cost)
#     for (drone,end,start) in itertools.combinations(range(n),3):
#         drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
#         truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[(drone+1)%n]])+t_pathcost((drone+1)%n,end,tsp)
#         T[start,drone,end]=max(drone_cost,truck_cost)
        
    for (i,j) in itertools.permutations(range(n),2):
        M[i,j]=float('inf')
        if i<j:
            if i+1==j:
                M[i,j]=t_cost(V[tsp[i]],V[tsp[j]])
            for k in range(i+1,j):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[i,j]=[k]
#         else:
#             for k in range(i+1,n):
#                 if M[i,j]>T[i,k,j]:
#                     M[i,j]=T[i,k,j]
#                     Mdrone[i,j]=[k]
#             for k in range(0,j):
#                 if M[i,j]>T[i,k,j]:
#                     M[i,j]=T[i,k,j]
#                     Mdrone[i,j]=[k]
#             M[n-1,0]=t_cost(V[tsp[n-1]],V[tsp[0]])

    depot=0
    for i in range(depot,depot+n+1):
        D[depot,i%n]=float('inf')
        if i==depot:
            D[depot,i]=0
            route[depot,i%n]=[[depot]]
        else:
            for j in range(depot,i):
                if j!=i%n:
                    d=M[j%n,i%n]+D[depot,j%n]
                    r=route[depot,j%n]+[[j%n,i%n]]
                    if D[depot,i%n]>d:
                            D[depot,i%n]=d
                            route[depot,i%n]=r
        
#     res=[D[i,i] for i in range(n)]
#     dep=res.index(min(res))
#     pathlen=len(route[dep,dep])
#     path=[]#通る順番（点の座標ではなくインデックスを表している点に注意）
#     for i in range(pathlen-1):
#         path=path+route[dep,dep][i+1]
#         if (route[dep,dep][i+1][0],route[dep,dep][i+1][1]) in Mdrone:
#             path=path+[Mdrone[route[dep,dep][i+1][0],route[dep,dep][i+1][1]]]
    
        
    
    return D[0,0]

In [28]:
import random
count=0
n=10
res=[]
while count <=1000:
    V = [(random.uniform(5,75),random.uniform(5,55)) for i in range(n)]
    lis =two_opt(V)
    if heuristic2(lis) < heuristic2original(lis):
        res.append(1)
    count =count+1
print(len(res))

1001
